# Sedaro Studies Example Notebook 
Demonstrates Sedaro Studies support via the sedaro python client using a Jupyter notebook.

# Introduction  
This notebook demonstrates how to use the new Studies support added to Sedaro. 
The core of this support is the new API python client object **SimStudy**.
**SimStudy** will generate and run a series of *SimJobs* in parallel up to account capacity limits. The remaining simjobs will be placed in a queue and will be execute when resources are available. 

Each *SimJob* of a **SimStudy** will set the random seed to a different value in order to generate different output results from the simulation.
The **Overrides** feature takes this a step further by providing a means to adjust any starting value of any model of the simulation using a set of pre-defined functions. In other words, each *SimJob* of a **SimStudy** will run a baseline Scenario branch with model parameters variations in order to observe their effects on performance results.

# Setup
Running this notebook requires the following:
- A Sedaro Account             --> https://www.sedaro.com
- A Sedaro API Token           --> https://www.sedaro.com/#/account
- The Wildfire Demo Branch ID  --> login --> select/click or create workspace --> select/click Project: [DEMO] WildFire --> select/click  Repositories: [DEMO] Wildfire *Scenarios* --> copy main branch ID via clipboard icon (TODO insert screen shot)
- The Wildfire Agent branch ID --> login --> select/click or create workspace --> select/click Project: [DEMO] WildFire --> select/click  Repositories: [DEMO] Wildfire *Vehicle* --> copy main branch ID via clipboard icon (TODO insert screen shot)
- Python 3.10+ installed       --> https://www.python.org
- Jupyter notebook or lab      --> https://jupyter.org

## Pip requirements
Create/activate a python venv if desired

In [1]:
#!python -m venv /path/to/new/virtual/environment

Activate it via this table
| Platform | Shell | Command to activate virtual environment |
| :- | :- | :- |
| POSIX | bash/zsh |  source <venv>/bin/activate |
| | fish |  source <venv>/bin/activate.fish |
| | csh/tcsh |  source <venv>/bin/activate.csh |
| | PowerShell |  <venv>/bin/Activate.ps1 |
| Windows | cmd.exe | C:\> <venv>\Scripts\activate.bat |
| | PowerShell | PS C:\> <venv>\Scripts\Activate.ps1 |

Required python modules:
> replace *python* with *python3* if needed 

In [2]:
#!python -m pip install -e sedaro pytest matplotlib pandas sweetviz

or

In [3]:
#!python3.10 -m pip install  sedaro pytest matplotlib pandas sweetviz

In [4]:
#!python3.10 -m pip install sweetviz IProgress flatten_json 'fuzzywuzzy[speedup]' python-Levenshtein

Run the following cells to test if all required python modules are installed correctly

Optional module needed by the stats() function if desired 

In [5]:
import sedaro
import yaml
import json
import matplotlib.pyplot as plt
import pandas as pd
import math
import numpy as np

In [6]:
# optional
import sweetviz as sv

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Important: Read Before Running

This notebook makes changes to agent and scenario branches indicated in the settings section. Ensure any changes to the target branches are saved prior to running this code. Sedaro recommends committing current changes and creating new branches in the target repositories to avoid loss of work.

This notebook also requires that you have previously generated an API key in the web UI. That key should be stored in a file called `secrets.json` in the same directory as this notebook with the following format:

```json
{
    "API_KEY": "<API_KEY>"
}
```

API keys grant full access to your repositories and should never be shared. If you think your API key has been compromised, you can revoke it in the user settings interface on the Sedaro website.



## Sedaro python client setup
Note: More information about the sedaro-python client can be found here: [https://github.com/sedaro/sedaro-python]


In the next cell, adjust the following variables as needed
- *Sedaro_api_host*
- *Sedaro_api_token*

In [7]:
Sedaro_api_host  = "http://localhost:80" # "api.sedaro.com"

# Set your API token value either directly or via loading a secrets file
# !! NOTE !!  Be careful not to check in your API Key into a source control repo  !! Note !!
secretPath = '/Users/sedaro/Documents/sedaro/sedaro-satellite/secrets.json'
with open(secretPath, 'r') as file:
    Sedaro_api_token = json.load(file)['API_KEY']
    
sedaroAPI = sedaro.SedaroApiClient(api_key=Sedaro_api_token, host=Sedaro_api_host)

## Load the wildfire scenario and wildfire agent branch data
Change the value of the *scenario_branch_id* and the *wildfire_agent_branch_id* in the next cell to the branch id noted above during the **Setup** section.
then run the following cell:

In [8]:
scenario_branch_id = "PKgBt4JWCmLVFBBzQBK3d8"
wildfire_scenario_branch = sedaroAPI.scenario(scenario_branch_id)

wildfire_agent_branch_id = "PKgBt3VQmHMZ8FzP9xYlsx"
wildfire_agent_branch = sedaroAPI.agent_template(wildfire_agent_branch_id)


In [11]:
import utils
wildfire_agent_paths = utils.AgentModelParametersOverridePaths(wildfire_scenario_branch, wildfire_agent_branch, agent_name='Wildfire')


In [13]:
wildfire_agent_paths.listPaths()

['Wildfire/Battery/capacity',
 'Wildfire/Battery/configurationType',
 'Wildfire/Battery/current',
 'Wildfire/Battery/curve',
 'Wildfire/Battery/disabled',
 'Wildfire/Battery/esr',
 'Wildfire/Battery/id',
 'Wildfire/Battery/idealMaxChargeCurrent',
 'Wildfire/Battery/idealMaxDischargeCurrent',
 'Wildfire/Battery/initialSoc',
 'Wildfire/Battery/maxChargeCurrent',
 'Wildfire/Battery/maxChargeCurrentOverride',
 'Wildfire/Battery/maxChargePower',
 'Wildfire/Battery/maxChargeVoltage',
 'Wildfire/Battery/maxDischargeCurrentOverride',
 'Wildfire/Battery/minSoc',
 'Wildfire/Battery/minSocOverride',
 'Wildfire/Battery/packs/0',
 'Wildfire/Battery/packs/1',
 'Wildfire/Battery/power',
 'Wildfire/Battery/powerProcessor',
 'Wildfire/Battery/soc',
 'Wildfire/Battery/type',
 'Wildfire/Battery/voc',
 'Wildfire/Battery/voltage',
 'Wildfire/Power Processor/activeDataMode',
 'Wildfire/Power Processor/activeLoadState',
 'Wildfire/Power Processor/availableSolarRootPower',
 'Wildfire/Power Processor/battery',

In [17]:
wildfire_agent_paths.findBestMatch("ratedMagneticMoment")

'Wildfire/MT-X/ratedMagneticMoment'

In [15]:
wildfire_agent_paths.findValueOf('Wildfire/data/inertia/2/2')

520.0

## Wildfire Monte Carlo Analysis: Understand uncertainty of average pointing error
In this study example, we will be looking at the effects of varying Agent Model paramaters and observe its effects on the Average pointing error

The model parameters we'll vary wil be:
- Rated Moment of magnetorquers
- Rated Torque of reaction wheels
- Dry Inertia Matrix 
- along with the effects of different random seeds the sensor models use.


### Optional: Load shared workspace to skip setup and running the study
TODO

### Setup
We'll reuse the scenario_branch_id from the Wildfire Tradespace example but if it was skipped,  Find or make a branch of the wildfire scenaro and record its branch ID value.

## Research
From Wikipedia:

> A reaction wheel (RW) is used primarily by spacecraft for three-axis attitude control, and does not require rockets or external applicators of torque. They provide a high pointing accuracy,[1]: 362  and are particularly useful when the spacecraft must be rotated by very small amounts, such as keeping a telescope pointed at a star.

> A magnetorquer or magnetic torquer (also known as a torque rod) is a satellite system for attitude control, detumbling, and stabilization built from electromagnetic coils. The magnetorquer creates a magnetic dipole that interfaces with an ambient magnetic field, usually Earth's, so that the counter-forces produced provide useful torque.


Example product sheets for Reacton Wheels and a magnetorquer Rod
> https://storage.googleapis.com/blue-canyon-tech-news/1/2023/04/ReactionWheels.pdf

> https://www.newspacesystems.com/wp-content/uploads/2022/07/NewSpace-Magnetorquer-Rod_V11.2.pdf

With this information, we'll adjust the magnetorquer **ratedMagneticMoment** and the reaction wheels **ratedTorque** model parameters along with dry mass interia.

### Find model parameter path 
Looking at the Wildfire scenario, it uses the wildfire template so lets search for the parameter paths we are interested in:

In [18]:
import utils
wildfire_agent_paths = utils.AgentModelParametersOverridePaths(wildfire_scenario_branch, wildfire_agent_branch, agent_name='Wildfire')

First search for **ratedMagneticMoment**

In [19]:
wildfire_agent_paths.findBestMatch("ratedMagneticMoment")

'Wildfire/MT-X/ratedMagneticMoment'

Next search for **ratedTorque**

In [20]:
wildfire_agent_paths.findBestMatch("ratedTorque")

'Wildfire/RW-X/ratedTorque'

Or we could search the Wildfire Agent path list directly using
> wildfire_agent_paths.listPaths()

In [21]:
[ path for path in wildfire_agent_paths.listPaths() if "ratedTorque" in path]

['Wildfire/RW-X/ratedTorque',
 'Wildfire/RW-Y/ratedTorque',
 'Wildfire/RW-Z/ratedTorque']

In [29]:
[ path for path in wildfire_agent_paths.listPaths() if "inertia" in path]

['Wildfire/RW-X/inertia',
 'Wildfire/RW-Y/inertia',
 'Wildfire/RW-Z/inertia',
 'Wildfire/Fuel Tank/inertia',
 'Wildfire/data/inertia/0/0',
 'Wildfire/data/inertia/0/1',
 'Wildfire/data/inertia/0/2',
 'Wildfire/data/inertia/1/0',
 'Wildfire/data/inertia/1/1',
 'Wildfire/data/inertia/1/2',
 'Wildfire/data/inertia/2/0',
 'Wildfire/data/inertia/2/1',
 'Wildfire/data/inertia/2/2']

#### Wildfire Agent parameters to vary to effect average pointing error 
- Magnetorquer rated magnetic moment
  - Blocks: MT-X, MT-Y, MT-Z
    - 'Wildfire/MT-X/ratedMagneticMoment'
    - 'Wildfire/MT-Y/ratedMagneticMoment'
    - 'Wildfire/MT-Z/ratedMagneticMoment'
- Reaction wheel Rated Torque
  - Blocks: RW-X, RW-Y, RW-Z
    - 'Wildfire/RW-X/ratedTorque'
    - 'Wildfire/RW-Y/ratedTorque'
    - 'Wildfire/RW-Z/ratedTorque'
- Dry Inertia Matrix 3x3
  - Block: root
    - *Wildfire/data/inertia/#/#*


#### Determine how to vary the parameters
First lets find the starting values of the selected parameters (or you can look at the web-client)

In [23]:
wildfire_agent_paths.findValueOf('Wildfire/RW-X/ratedTorque')

0.25


> Which is the same result for **Wildfire/RW-Y/ratedTorque** and **Wildfire/RW-Z/ratedTorque** 


In [24]:

wildfire_agent_paths.findValueOf('Wildfire/MT-X/ratedMagneticMoment')

10.0

> Which is the same result for  **Wildfire/MT-Y/ratedMagneticMoment**  and  **Wildfire/MT-Z/ratedMagneticMoment**


Note: Dry inertia matrix is diagonal 


In [26]:
wildfire_agent_paths.findValueOf('Wildfire/data/inertia/0/0')


270.0

In [27]:
wildfire_agent_paths.findValueOf('Wildfire/data/inertia/1/1')

420.0

In [28]:
wildfire_agent_paths.findValueOf('Wildfire/data/inertia/2/2')

520.0

Calculate the RMS of the dry mass inertia. We will use a max of 2% of it to adjust the dry mass matrix parameters. 

In [32]:
inertia_0_0 = wildfire_agent_paths.findValueOf('Wildfire/data/inertia/0/0')
inertia_1_1 = wildfire_agent_paths.findValueOf('Wildfire/data/inertia/1/1')
inertia_2_2 = wildfire_agent_paths.findValueOf('Wildfire/data/inertia/2/2')
mean = (inertia_0_0 + inertia_1_1 + inertia_2_2)/3
inertia_0_0_delta = inertia_0_0 - mean
inertia_1_1_delta = inertia_1_1 - mean
inertia_2_2_delta = inertia_2_2 - mean
mean_square = (inertia_0_0_delta*inertia_0_0_delta + inertia_1_1_delta*inertia_1_1_delta + inertia_2_2_delta*inertia_2_2_delta)/3
root_mean_square = math.sqrt(mean_square)
root_mean_square*0.02


2.0548046676563256

#### Create Override dict


In [34]:
monte_carlo_overrides_dict = {
  "name": "Monte Carlo Example",
  "variables": [
    {
      "name": "inertia_root_mean_square_two_percent",
      "equals": 2.0548046676563256
    }
  ],
  "overrides": [ 
      {
          "path": "Wildfire/RW-X/ratedTorque",
          "fn": "normalvariate",
          "mu": "x",
          "sigma": 0.25*0.02
      },
      {
          "path": "Wildfire/RW-Y/ratedTorque",
          "fn": "normalvariate",
          "mu": "x",
          "sigma": 0.25*0.02
      },
      {
          "path": "Wildfire/RW-Z/ratedTorque",
          "fn": "normalvariate",
          "mu": "x",
          "sigma": 0.25*0.02
      },
      {
          "path": "Wildfire/MT-X/ratedMagneticMoment",
          "fn": "gauss",
          "mu": "x",
          "sigma": 10.0*0.02
      },
      {
          "path": "Wildfire/MT-Y/ratedMagneticMoment",
          "fn": "gauss",
          "mu": "x",
          "sigma": 10.0*0.02
      },
      {
          "path": "Wildfire/MT-Z/ratedMagneticMoment",
          "fn": "gauss",
          "mu": "x",
          "sigma": 10.0*0.02
      },
      {
          "path": "Wildfire/data/inertia/0/0",
          "fn": "normalvariate",
          "mu": "x",
          "sigma": "inertia_root_mean_square_two_percent"
      },
      {
          "path": "Wildfire/data/inertia/0/1",
          "fn": "normalvariate",
          "mu": "x",
          "sigma": "inertia_root_mean_square_two_percent"          
      },
      {
          "path": "Wildfire/data/inertia/0/2",
          "fn": "normalvariate",
          "mu": "x",
          "sigma": "inertia_root_mean_square_two_percent"          
      },      
      {
          "path": "Wildfire/data/inertia/1/0",
          "fn": "normalvariate",
          "mu": "x",
          "sigma": "inertia_root_mean_square_two_percent"          
      },      
      {
          "path": "Wildfire/data/inertia/1/1",
          "fn": "normalvariate",
          "mu": "x",
          "sigma": "inertia_root_mean_square_two_percent"
      },
      {
          "path": "Wildfire/data/inertia/1/2",
          "fn": "normalvariate",
          "mu": "x",
          "sigma": "inertia_root_mean_square_two_percent"          
      },  
      {
          "path": "Wildfire/data/inertia/2/0",
          "fn": "normalvariate",
          "mu": "x",
          "sigma": "inertia_root_mean_square_two_percent"          
      },
      {
          "path": "Wildfire/data/inertia/2/1",
          "fn": "normalvariate",
          "mu": "x",
          "sigma": "inertia_root_mean_square_two_percent"
      },          
      {
          "path": "Wildfire/data/inertia/2/2",
          "fn": "normalvariate",
          "mu": "x",
          "sigma": "inertia_root_mean_square_two_percent"
      },

  ]
}

In [35]:
#monte_carlo_overrides_block = wildfire_scenario_branch.OverrideSet.create(**monte_carlo_overrides_dict)

In [37]:
tradespace_overrides_block = (wildfire_scenario_branch.OverrideSet.get_last())

In [36]:
monte_carlo_overrides_block


OverrideSet(
   disabled=False
   id='PKqTMXLvGrPQqyPQgrVJX7'
   name='Monte Carlo Example'
   overrides=[{'agent_key': None, 'fn': 'normalvariate', 'mu': 'x', 'path': 'Wildfire/RW-X/ratedTorque', 'sigma': 0.005}, {'agent_key': None, 'fn': 'normalvariate', 'mu': 'x', 'path': 'Wildfire/RW-Y/ratedTorque', 'sigma': 0.005}, {'agent_key': None, 'fn': 'normalvariate', 'mu': 'x', 'path': 'Wildfire/RW-Z/ratedTorque', 'sigma': 0.005}, {'agent_key': None, 'fn': 'gauss', 'mu': 'x', 'path': 'Wildfire/MT-X/ratedMagneticMoment', 'sigma': 0.2}, {'agent_key': None, 'fn': 'gauss', 'mu': 'x', 'path': 'Wildfire/MT-Y/ratedMagneticMoment', 'sigma': 0.2}, {'agent_key': None, 'fn': 'gauss', 'mu': 'x', 'path': 'Wildfire/MT-Z/ratedMagneticMoment', 'sigma': 0.2}, {'agent_key': None, 'fn': 'normalvariate', 'mu': 'x', 'path': 'Wildfire/data/inertia/0/0', 'sigma': 'inertia_root_mean_square_two_percent'}, {'agent_key': None, 'fn': 'normalvariate', 'mu': 'x', 'path': 'Wildfire/data/inertia/0/1', 'sigma': 'inertia_r

Record the override block ID

In [39]:
monte_carlo_overrides_id = monte_carlo_overrides_block.id

#### Create and run Study

In [ ]:
create_study_resource_url = f'/simulations/branches/{scenario_branch_id}/control/study/'
new_studyjob = sedaroAPI.request.post(  create_study_resource_url,
                                        body={
                                            "iterations": 6,
                                            "override_id": monte_carlo_overrides_id
                                            })
[ (study['id'], study['status']) for study in sedaroAPI.request.get(  create_study_resource_url) ]

In [ ]:
new_studyjob

### Check on Study Status

### Load the results of the Study SimJobs
Load the the Study Results then load the result data from all of its  simulation set. We'll reduce the data set to 1000 points each for this example.

### Analyze Results